In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn



In [ ]:
df = pd.read_csv('/dataset.csv')
df

In [ ]:
df.isna().sum()


In [ ]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)


In [ ]:
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]
df

In [ ]:
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("parking_stream.csv", index=False)


In [ ]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: bool
    VehicleType: str

In [ ]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)


In [ ]:
@pw.udf
def traffic_to_level(traffic: str) -> int:
    return {"low": 0, "average": 1, "high": 2}.get(traffic, 1)

@pw.udf
def vehicle_factor(vtype: str) -> int:
    return {"cycle": 0, "bike": 1, "car": 2, "truck": 3}.get(vtype, 1)


fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),
    OccupancyRate = data.Occupancy / data.Capacity,
    TrafficLevel = traffic_to_level(data.TrafficConditionNearby),
    VehicleFactor = vehicle_factor(data.VehicleType),
    IsSpecialDay = data.IsSpecialDay,
    QueueLength = data.QueueLength,
    lot = data.SystemCodeNumber,
    instance = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d") + "_" + data.SystemCodeNumber
)


In [ ]:
@pw.udf
def compute_demand(occ: float, queue: float, traffic: float, special: bool, vehicle: float) -> float:
    α, β, γ, δ, ε = 5, 0.4, 1.5, 2, 1
    demand = α * occ + β * queue - γ * traffic + δ * int(special) + ε * vehicle
    return demand


In [ ]:
@pw.udf
def compute_price(base: float, demand: float, max_demand: float = 50.0, λ: float = 0.2) -> float:
    norm_demand = min(demand / max_demand, 1.0)
    raw_price = base * (1 + λ * norm_demand)
    return max(0.5 * base, min(raw_price, 2.0 * base))


In [ ]:

import datetime

reduced = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.instance,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.OccupancyRate),
        queue_sum = pw.reducers.sum(pw.this.QueueLength),
        traffic_sum = pw.reducers.sum(pw.this.TrafficLevel),
        vehicle_sum = pw.reducers.sum(pw.this.VehicleFactor),
        special_any = pw.reducers.max(pw.this.IsSpecialDay),
        n = pw.reducers.count(),
        lot = pw.reducers.min(pw.this.lot)
    )
    .with_columns(
        occ_avg = pw.this.occ_sum / pw.this.n,
        queue_avg = pw.this.queue_sum / pw.this.n,
        traffic_avg = pw.this.traffic_sum / pw.this.n,
        vehicle_avg = pw.this.vehicle_sum / pw.this.n
    )
)


In [ ]:
delta_window = reduced.with_columns(
    demand = compute_demand(
        reduced.occ_avg,
        reduced.queue_avg,
        reduced.traffic_avg,
        reduced.special_any,
        reduced.vehicle_avg
    )
)

delta_window = delta_window.with_columns(
    price = compute_price(10.0, delta_window.demand)
)

In [ ]:
pn.extension()

def price_plotter(source):
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource
    from bokeh.palettes import Category10_10 as palette

    df = pd.DataFrame(source.data)
    df["t"] = pd.to_datetime(df["t"])
    df["lot"] = df["lot"].astype(str)

    fig = figure(
        height=500,
        width=900,
        title="Daily Pricing Across Parking Lots",
        x_axis_type="datetime"
    )

    for i, lot in enumerate(sorted(df["lot"].unique())):
        df_lot = df[df["lot"] == lot]
        source_lot = ColumnDataSource(df_lot)

        fig.line("t", "price", source=source_lot,
                 legend_label=f"Lot {lot}",
                 line_width=2,
                 color=palette[i % len(palette)])

    fig.legend.location = "top_left"
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"


    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


In [ ]:
pw.io.jsonlines.write(
    delta_window.select(pw.this.t,pw.this.lot,pw.this.price),
    "pricing_output.jsonl",
)
pw.run()

In [ ]:
from bokeh.plotting import figure, output_file, save , output_file
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from bokeh.palettes import Category10_10 as palette
from datetime import datetime
from bokeh.models import Legend

df = pd.read_json("pricing_output.jsonl", lines=True)

df["t"] = pd.to_datetime(df["t"])
df["lot"] = df["lot"].astype(str)

fig = figure(
    height=500,
    width=900,
    title="Daily Pricing Across Parking Lots",
    x_axis_type="datetime"
)

lots = sorted(df["lot"].unique())
for i, lot in enumerate(lots):
    df_lot = df[df["lot"] == lot]
    source = ColumnDataSource(df_lot)

    fig.line("t", "price", source=source, legend_label=f"Lot {lot}",
             color=palette[i % len(palette)], line_width=2)

fig.xaxis.axis_label = "Time"
fig.yaxis.axis_label = "Price ($)"
fig.add_layout(fig.legend[0], 'right')
fig.legend.label_text_font_size = "8pt"

output_file("lot_wise_comparision.html")
save(fig)


In [ ]:
from bokeh.palettes import Category10_10
df = pd.read_json("pricing_output.jsonl", lines=True)
df["t"] = pd.to_datetime(df["t"])
df["lot"] = df["lot"].astype(str)

plots = []
palette = Category10_10

for i, lot in enumerate(sorted(df["lot"].unique())):
    df_lot = df[df["lot"] == lot]
    source = ColumnDataSource(df_lot)

    p = figure(
        height=300,
        width=800,
        title=f"Daily Price: Lot {lot}",
        x_axis_type="datetime"
    )
    p.line("t", "price", source=source,
           line_width=2, color=palette[i % len(palette)])
    p.scatter("t", "price", source=source, size=5, marker="circle", color="black")

    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    plots.append(p)

output_file("all_lots_prices_model2.html")
save(column(*plots))
